In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime 
import time
import radix
import gpxpy
import gpxpy.gpx
import requests
import math
import json

%matplotlib inline

In [3]:
DataFrame1 = pd.read_csv('track_points.csv', sep=",", usecols=(0,1))

In [4]:
DataFrame1.head()

,X,Y
0,-123.256691,44.587662
1,-123.256691,44.587662
2,-123.256691,44.587654
3,-123.256699,44.587643
4,-123.256714,44.587627


In [5]:
DataFrame1.iloc[0:]

,X,Y
0,-123.256691,44.587662
1,-123.256691,44.587662
2,-123.256691,44.587654
3,-123.256699,44.587643
4,-123.256714,44.587627
...,...,...
18370,-123.263397,44.587345
18371,-123.263428,44.587334
18372,-123.263451,44.587311
18373,-123.263458,44.587288


In [6]:
len(DataFrame1)

18375

In [174]:
# a small dataframe for testing 
df_len = len(DataFrame1)
# DataFrame2 = pd.DataFrame()
small_data = list()
for i in range(df_len):
    denominator = math.ceil(df_len/50)
    if i % denominator == 0:
        small_data.append(DataFrame1.iloc[i])
      
DataFrame2 = pd.DataFrame(small_data)
DataFrame2.head()

,X,Y
0,-123.256691,44.587662
368,-123.261696,44.598076
736,-123.264656,44.612839
1104,-123.273796,44.624504
1472,-123.274628,44.630474


In [175]:
DataFrame2 = DataFrame2.reset_index(drop=True)
DataFrame2.head()

,X,Y
0,-123.256691,44.587662
1,-123.261696,44.598076
2,-123.264656,44.612839
3,-123.273796,44.624504
4,-123.274628,44.630474


In [176]:
# for i in range(len(DataFrame2)):
#     DataFrame2.iloc[i][3] = None
# del DataFrame2["Z"]
# del DataFrame2["streetName"]
DataFrame2["streetName"]=""
DataFrame2.head()

,X,Y,streetName
0,-123.256691,44.587662,
1,-123.261696,44.598076,
2,-123.264656,44.612839,
3,-123.273796,44.624504,
4,-123.274628,44.630474,


In [138]:
DataFrame2 = DataFrame2.iloc[0:10]

In [139]:
print(DataFrame2.iloc[0:5])

            X          Y streetName
0 -123.256691  44.587662           
1 -123.264595  44.612835           
2 -123.274559  44.630478           
3 -123.295586  44.636280           
4 -123.278564  44.662323           


In [177]:
DataFrame2.head()

,X,Y,streetName
0,-123.256691,44.587662,
1,-123.261696,44.598076,
2,-123.264656,44.612839,
3,-123.273796,44.624504,
4,-123.274628,44.630474,


In [141]:
DataFrame2.iloc[0][0]

-123.25669099999999

In [142]:
DataFrame1.index[-1]

18374

In [143]:
DataFrame2.index[-1]

9

In [144]:
q = 0
M = list()
for index, item in DataFrame2.iterrows():
    row, col, streetAddress = item
#     print(row, col)
    l = [row, col]
#     print(l)
    M.append(l)
    q = q + 1
    if q > 3:
        break
print(M)
for item in M:
    print(item)

[[-123.25669099999999, 44.587662], [-123.264595, 44.612834899999996], [-123.27455900000001, 44.630477899999995], [-123.29558600000001, 44.63628]]
[-123.25669099999999, 44.587662]
[-123.264595, 44.612834899999996]
[-123.27455900000001, 44.630477899999995]
[-123.29558600000001, 44.63628]


In [131]:
n = 0
for i,row in DataFrame2.iterrows():
    lon, lat, streetName = row
    web_input = {'lon':str(lon), 
                  'lat':str(lat), 
                  'state':"or", 
                  'apikey':"06b0930203314ecc90082f30a9e65753", 
                 'format':"json",
                 'notStore':"false",
                 'includeHeader':"false",
                 'version':"4.10"}
    website = requests.get('https://geoservices.tamu.edu/Services/ReverseGeocoding/WebService/v04_01/Rest/?',
                           params=web_input)
    
    print(lon,lat)
    print(website.content)
    json_read = json.loads(website.content)
    l = str(website.content).split(',')
    print(str(website.content).split(',')[5])
    
    n = n + 1
    if n > 0:
        break
    
json_read['StreetAddresses'][0]['StreetAddress']

-123.25669099999999 44.587662
b'{\r\n\t"TransactionId" : "5ac1f43d-b816-42c0-af8b-ac479bf59135",\r\n\t"Version"   : "4.01",\r\n\t"QueryStatusCode" : "Success",\r\n\t"TimeTaken" : "15.6131",\r\n\t"Exception" : "",\r\n\t"ErrorMessage" : "",\r\n\t"StreetAddresses" : \r\n\t[\r\n\t{\r\n\t\t"TransactionId" : "5ac1f43d-b816-42c0-af8b-ac479bf59135",\r\n\t\t"Version"   : "4.01",\r\n\t\t"QueryStatusCode" : "Success",\r\n\t\t"TimeTaken" : "0",\r\n\t\t"Exception" : "",\r\n\t\t"ErrorMessage" : "",\r\n\t\t"APN" : "11526AD02202",\r\n\t\t"StreetAddress" : "922 NW Circle Blvd",\r\n\t\t"City" : "Corvallis",\r\n\t\t"State" : "OR",\r\n\t\t"Zip" : "97330",\r\n\t\t"ZipPlus4" : "1410"\r\n\t}\r\n\t]\r\n}\r\n\r\n'
\r\n\t"ErrorMessage" : ""


'922 NW Circle Blvd'

In [178]:
def get_address(df, index, global_index, one_or_zero):
    print("df index:")
    print("index ---------- ", index)
    df_len = len(df)
    for i in range(df_len):
        print(df.index[i])
    print()
    lon = df.iloc[index][0]
    lat = df.iloc[index][1]
    web_input = {'lon':str(lon), 
                  'lat':str(lat), 
                  'state':"or", 
                  'apikey':"68971949e5f24edba97fb29083c1a604", 
                 'format':"json",
                 'notStore':"false",
                 'version':"4.10"}
    website = requests.get('https://geoservices.tamu.edu/Services/ReverseGeocoding/WebService/v04_01/Rest/?',
                           params=web_input)
    json_read = json.loads(website.content)
    print("GLOBAL_INDEX:", global_index)
    print("!!!!!!@@@@@@@@@################QueryStatusCode:", json_read['QueryStatusCode'])
    idx_cp = index
    print(f"IDX_CP AT FIRST ^^^^^^^^^^ : {idx_cp}")
    if one_or_zero == 1:
        while json_read['QueryStatusCode'] != "Success":
            print("FAILURE index 1", index)
            print("QueryStatus Failure!!!!!!!!!!!!!!!!!!!!1")
            DataFrame2.at[global_index,'streetName'] = "Not Available" # global
            idx_cp = idx_cp + 1
            lon = df.iloc[idx_cp][0]
            lat = df.iloc[idx_cp][1]
            web_input = {'lon':str(lon), 
                      'lat':str(lat), 
                      'state':"or", 
                      'apikey':"68971949e5f24edba97fb29083c1a604", 
                     'format':"json",
                     'notStore':"false",
                     'version':"4.10"}
            website = requests.get('https://geoservices.tamu.edu/Services/ReverseGeocoding/WebService/v04_01/Rest/?',
                                   params=web_input)
            json_read = json.loads(website.content)
    else:
        while json_read['QueryStatusCode'] != "Success":
            print("FAILURE index 0", index)
            print("QueryStatus Failure!!!!!!!!!!!!!!!!!!!!1")
            idx_cp = idx_cp - 1 
            lon = df.iloc[idx_cp][0]
            lat = df.iloc[idx_cp][1]
            web_input = {'lon':str(lon), 
                      'lat':str(lat), 
                      'state':"or", 
                      'apikey':"68971949e5f24edba97fb29083c1a604", 
                     'format':"json",
                     'notStore':"false",
                     'version':"4.10"}
            website = requests.get('https://geoservices.tamu.edu/Services/ReverseGeocoding/WebService/v04_01/Rest/?',
                                   params=web_input)
            json_read = json.loads(website.content)
    print("index cp", idx_cp)
    print(json_read)
    address = str(json_read['StreetAddresses'][0]['StreetAddress'])
#     DataFrame2.at[global_index,'streetName'] = address # global
    return address

In [133]:
print(get_address(DataFrame2, 0, 1))

index 0
df index:
0
1
2
3
4
5
6
7
8
9

{'TransactionId': '0394a810-29e7-4dc3-9508-e142f100e2f8', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '125.9974', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '0394a810-29e7-4dc3-9508-e142f100e2f8', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '0', 'Exception': '', 'ErrorMessage': '', 'APN': '11526AD02202', 'StreetAddress': '922 NW Circle Blvd', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '1410'}]}
922 NW Circle Blvd


In [179]:
DataFrame2.head(10)

,X,Y,streetName
0,-123.256691,44.587662,
1,-123.261696,44.598076,
2,-123.264656,44.612839,
3,-123.273796,44.624504,
4,-123.274628,44.630474,
5,-123.284256,44.634384,
6,-123.295631,44.636330,
7,-123.294273,44.648167,
8,-123.278419,44.662510,
9,-123.261612,44.673931,


In [206]:
s = '12abc d405 '
d = "     213rm g sdvs "
# result = ''.join([i for i in s if not i.isdigit()])
result2 = "".join([i for i in d if not i.isdigit()])
result2 = result2.strip()
print(result2)

rm g sdvs


In [184]:
turn_pts = list()
def binarySearch(df):
    print()
    print()
    print()
    print()
    print()
    print(DataFrame2)
    print()
    print()
    print()
    print()
    print()
    print("start a new bs")
    # df = df.reset_index(drop=True)
    print("size of the dataframeeeeee: ", len(df))
    start = df.index[0]
    print("start:",start)
    end = df.index[-1]
    print(f"HERE WE ARE AT end = [{end}] = df.index[-1]----------------")
    mid = math.ceil((start + end) / 2)
    print(f"start: {start}, mid: {mid}, end: {end}")
    local_start = 0
    local_end = len(df) - 1
    local_mid = math.ceil((local_start + local_end) / 2)
    # find start point address
    if DataFrame2.iloc[start]['streetName'] == "":
        print("It's a start")
        start_address = get_address(df, local_start, start, 1)    # something is wrong here, why local_start
    else:
        start_address = DataFrame2.iloc[start]['streetName']
    
    # find mid point address 
    if DataFrame2.iloc[mid]['streetName'] == "":
        print("It's a mid")
        mid_address = get_address(df, local_mid, mid, 1)
    else:
        mid_address = DataFrame2.iloc[mid]['streetName']
    
    # find end point address 
    if DataFrame2.iloc[end]['streetName'] == "":
        print("It's a end")
        end_address = get_address(df, local_end, end, 0)
    else:
        end_address = DataFrame2.iloc[end]['streetName']
        
    # strip numeric characters from addresses
    start_street = ''.join([i for i in start_address if not i.isdigit()])
    start_street = start_street.strip()
    
    mid_street = ''.join([i for i in mid_address if not i.isdigit()])
    mid_street = mid_street.strip()
    
    end_street = ''.join([i for i in end_address if not i.isdigit()])
    end_street = end_street.strip()
    
    # base case
    if len(df) <= 2:
        print("<= 2, I'm returning")
        for index, item in df.iterrows():
            row, col, streetAddress = item
            l = [row, col]
            turn_pts.append(l)
        print("turning points:", turn_pts)
        return None
    
    elif start_street == mid_street and mid_street == end_street:
        
        if end - start < math.ceil(len(df) / 1000):  # 
            print("I'm returning...")
            print("turning points:", turn_pts)
            return None
        else:
            print("from start to mid-----------")
            print(df.iloc[local_start:local_mid+1])
            print(f"SIZE OF SMALL DF: ---- {len(df)} ---- after start:mid+1")
            binarySearch(df.iloc[local_start:local_mid+1])
            print("from mid to end=============")
            print(df.iloc[local_mid:])
            print(f"SIZE OF SMALL DF: ---- {len(df)} ----")
            binarySearch(df.iloc[local_mid:])
            
    else:
        print(f"SIZE OF SMALL DF: ---- {len(df)} ---- in BIG ELSE")
        print(f"our STARTING POINT NOW IS {start} MID is {mid}")
        
        binarySearch(df.iloc[local_start:local_mid+1])
        binarySearch(df.iloc[local_mid:])

In [171]:
binarySearch(DataFrame2)







             X          Y          streetName
0  -123.256691  44.587662  922 NW Circle Blvd
1  -123.261696  44.598076                    
2  -123.264656  44.612839                    
3  -123.273796  44.624504                    
4  -123.274628  44.630474                    
5  -123.284256  44.634384                    
6  -123.295631  44.636330                    
7  -123.294273  44.648167                    
8  -123.278419  44.662510                    
9  -123.261612  44.673931                    
10 -123.243546  44.683239                    
11 -123.257011  44.696350                    
12 -123.274811  44.699432                    
13 -123.291039  44.706707                    
14 -123.305023  44.726711                    
15 -123.296165  44.741398                    
16 -123.270699  44.742970                    
17 -123.246521  44.744038                    
18 -123.223228  44.747009                    
19 -123.227470  44.764683                    
20 -123.235352  44.780777    

{'TransactionId': 'cbdc68a1-19b8-49ba-b6bf-7f453092f4dc', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '31.2462', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': 'cbdc68a1-19b8-49ba-b6bf-7f453092f4dc', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '15.613', 'Exception': '', 'ErrorMessage': '', 'APN': '11504A000200', 'StreetAddress': '37072 Soap Creek Rd', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9333'}]}
SIZE OF SMALL DF: ---- 14 ---- in BIG ELSE
our STARTING POINT NOW IS 0 MID is 7





             X          Y            streetName
0  -123.256691  44.587662    922 NW Circle Blvd
1  -123.261696  44.598076                      
2  -123.264656  44.612839                      
3  -123.273796  44.624504                      
4  -123.274628  44.630474                      
5  -123.284256  44.634384                      
6  -123.295631  44.636330                      
7  -123.294273  44.648167   37072 Soap 

{'TransactionId': '53befecf-b1d1-49b6-9ca6-5d715dd06015', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '296.8691', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '53befecf-b1d1-49b6-9ca6-5d715dd06015', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '281.26', 'Exception': '', 'ErrorMessage': '', 'APN': '11523DB00100', 'StreetAddress': '3045 NW Highland Dr', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9701'}]}
SIZE OF SMALL DF: ---- 3 ---- in BIG ELSE
our STARTING POINT NOW IS 0 MID is 1





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504                            
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384                            
6  -123.295631  44.636330                    

{'TransactionId': 'fe72dfb7-4465-40f6-8eb9-e82b60fdc8dd', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '264.9466', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': 'fe72dfb7-4465-40f6-8eb9-e82b60fdc8dd', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '258.9486', 'Exception': '', 'ErrorMessage': '', 'APN': '11510D003800', 'StreetAddress': '5432 NW Crescent Valley Dr', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9723'}]}
SIZE OF SMALL DF: ---- 3 ---- in BIG ELSE
our STARTING POINT NOW IS 2 MID is 3





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384                            
6  -123.295631  44.636330           

{'TransactionId': '6ba01d8e-85e3-43dd-a071-10cf397fe31c', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '468.7503', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '6ba01d8e-85e3-43dd-a071-10cf397fe31c', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '468.7503', 'Exception': '', 'ErrorMessage': '', 'APN': '11510B003500', 'StreetAddress': '2471 NW Michelle Dr', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9267'}]}
SIZE OF SMALL DF: ---- 4 ---- in BIG ELSE
our STARTING POINT NOW IS 4 MID is 6





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384                            
6  -123.295631  44.636330         2471 NW M

...

{'TransactionId': '529c826f-8b80-440d-b14f-830451f307f8', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '46.9972', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '529c826f-8b80-440d-b14f-830451f307f8', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '40.9996', 'Exception': '', 'ErrorMessage': '', 'APN': '104210001102', 'StreetAddress': '4736 NW Independence Hwy', 'City': 'Tangent', 'State': 'OR', 'Zip': '97321', 'ZipPlus4': '9102'}]}
SIZE OF SMALL DF: ---- 13 ---- in BIG ELSE
our STARTING POINT NOW IS 37 MID is 43





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384  2800 NW Sulphur Springs Rd
6  -123.295631  44.636330         2471 

{'TransactionId': '36a23cc1-029a-489b-a7d0-75e6989984d7', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '46.8803', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '36a23cc1-029a-489b-a7d0-75e6989984d7', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '31.2535', 'Exception': '', 'ErrorMessage': '', 'APN': '9423C0 3200', 'StreetAddress': '11350 CHURCH ST', 'City': 'Independence', 'State': 'OR', 'Zip': '97351', 'ZipPlus4': '9723'}]}
SIZE OF SMALL DF: ---- 3 ---- in BIG ELSE
our STARTING POINT NOW IS 37 MID is 38





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384  2800 NW Sulphur Springs Rd
6  -123.295631  44.636330         2471 NW Mic

{'TransactionId': '80197178-19c9-4297-b65f-7bb4b658ff36', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '31.2588', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': '80197178-19c9-4297-b65f-7bb4b658ff36', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '31.2588', 'Exception': '', 'ErrorMessage': '', 'APN': '104160000480', 'StreetAddress': '7299 NW Independence Hwy', 'City': 'Tangent', 'State': 'OR', 'Zip': '97321', 'ZipPlus4': '9353'}]}
SIZE OF SMALL DF: ---- 4 ---- in BIG ELSE
our STARTING POINT NOW IS 40 MID is 42





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384  2800 NW Sulphur Springs Rd
6  -123.295631  44.636330         2471 N

{'TransactionId': 'c4e8de16-71fc-456e-845a-dabde9f6c8bb', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '176.7669', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': 'c4e8de16-71fc-456e-845a-dabde9f6c8bb', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '168.7607', 'Exception': '', 'ErrorMessage': '', 'APN': '114060002001', 'StreetAddress': '7040 NE Pettibone Dr', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9612'}]}
SIZE OF SMALL DF: ---- 7 ---- in BIG ELSE
our STARTING POINT NOW IS 43 MID is 46





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384  2800 NW Sulphur Springs Rd
6  -123.295631  44.636330         2471 N

{'TransactionId': 'e7ba04c3-cf14-48d6-bef9-95aa75952fab', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '222.5286', 'Exception': '', 'ErrorMessage': '', 'StreetAddresses': [{'TransactionId': 'e7ba04c3-cf14-48d6-bef9-95aa75952fab', 'Version': '4.01', 'QueryStatusCode': 'Success', 'TimeTaken': '216.527', 'Exception': '', 'ErrorMessage': '', 'APN': '11512C000500', 'StreetAddress': '5210 NE Elliott Cir', 'City': 'Corvallis', 'State': 'OR', 'Zip': '97330', 'ZipPlus4': '9432'}]}
SIZE OF SMALL DF: ---- 4 ---- in BIG ELSE
our STARTING POINT NOW IS 46 MID is 48





             X          Y                  streetName
0  -123.256691  44.587662          922 NW Circle Blvd
1  -123.261696  44.598076         3045 NW Highland Dr
2  -123.264656  44.612839  4160 NW Crescent Valley Dr
3  -123.273796  44.624504  5432 NW Crescent Valley Dr
4  -123.274628  44.630474  1635 NW Sulphur Springs Rd
5  -123.284256  44.634384  2800 NW Sulphur Springs Rd
6  -123.295631  44.636330         2471 NW 

In [172]:
DataFrame2

,X,Y,streetName
0,-123.256691,44.587662,922 NW Circle Blvd
1,-123.261696,44.598076,3045 NW Highland Dr
2,-123.264656,44.612839,4160 NW Crescent Valley Dr
3,-123.273796,44.624504,5432 NW Crescent Valley Dr
4,-123.274628,44.630474,1635 NW Sulphur Springs Rd
5,-123.284256,44.634384,2800 NW Sulphur Springs Rd
6,-123.295631,44.636330,2471 NW Michelle Dr
7,-123.294273,44.648167,37072 Soap Creek Rd
8,-123.278419,44.662510,37524 Soap Creek Rd
9,-123.261612,44.673931,28120 Beef Barn Rd
